Goal: SQRA try differet merge and cut marks and visualize resulting differences.

Basic set up:
- PT with FullGrid 

In [1]:
import sys
import os
from time import time

try:
    os.chdir(r"/home/zupanhana/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/zupanhana/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\\HANA\\phD\\PAPER_2022\\molecularRotationalGrids")
    sys.path.append(r"D:\\HANA\\phD\\PAPER_2022\\molecularRotationalGrids")


In [2]:
import numpy as np
from scipy import sparse
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from molgri.molecules.parsers import XVGParser
from molgri.molecules.transitions import SQRA
from molgri.molecules.transitions import DecompositionTool
from molgri.paths import PATH_OUTPUT_AUTOSAVE, PATH_OUTPUT_ENERGIES, PATH_OUTPUT_AUTOSAVE
from molgri.plotting.transition_plots import PlotlyTransitions

In [11]:
def run_base_sqra(pt_identifier, grid_identifier, output_path):
    t1 = time()
    # load input files
    all_volumes = np.load(f"{PATH_OUTPUT_AUTOSAVE}{grid_identifier}_volumes.npy")
    all_surfaces = sparse.load_npz(f"{PATH_OUTPUT_AUTOSAVE}{grid_identifier}_borders_array.npz")
    all_distances = sparse.load_npz(f"{PATH_OUTPUT_AUTOSAVE}{grid_identifier}_distances_array.npz")
    
    my_parsed = XVGParser(f"{PATH_OUTPUT_ENERGIES}{pt_identifier}-{grid_identifier}_energy.xvg")
    energies = my_parsed.get_parsed_energy().get_energies("Potential")
    
    sqra = SQRA(energies=energies, volumes=all_volumes, distances=all_distances, surfaces=all_surfaces)
    rate_matrix = sqra.get_rate_matrix(D =1, T=273)
    sparse.save_npz(output_path, rate_matrix)

    t2 = time()
    return rate_matrix.shape, t2-t1

def run_cut_merge(grid_identifier, pt_identifier, rate_matrix_path, lower_limit, upper_limit, output_path, output_index, T=273):
    t1 = time()
    my_matrix = sparse.load_npz(rate_matrix_path)
    all_volumes = np.load(f"{PATH_OUTPUT_AUTOSAVE}{grid_identifier}_volumes.npy")
    all_surfaces = sparse.load_npz(f"{PATH_OUTPUT_AUTOSAVE}{grid_identifier}_borders_array.npz")
    all_distances = sparse.load_npz(f"{PATH_OUTPUT_AUTOSAVE}{grid_identifier}_distances_array.npz")
    
    my_parsed = XVGParser(f"{PATH_OUTPUT_ENERGIES}{pt_identifier}-{grid_identifier}_energy.xvg")
    energies = my_parsed.get_parsed_energy().get_energies("Potential")

    sqra = SQRA(energies=energies, volumes=all_volumes, distances=all_distances, surfaces=all_surfaces)
    rate_matrix, index_list = sqra.cut_and_merge(my_matrix, T=T, lower_limit=lower_limit,
        upper_limit=upper_limit)

    # saving to file
    sparse.save_npz(output_path, rate_matrix)
    np.save(output_index, np.array(index_list, dtype=object))
    t2 = time()
    print(t2-t1)

def run_get_eigenval_eigenvec(rate_matrix_path, output_eigenval, output_eigenvec):
    t1 = time()
    # loading
    my_matrix = sparse.load_npz(rate_matrix_path)

    # calculation
    dt = DecompositionTool(my_matrix,is_msm=False)
    all_eigenval, all_eigenvec = dt.get_decomposition(sigma = 0, which = "LM", tol = 1e-5, maxiter = 100000)

    # saving to file
    np.save(output_eigenval,np.array(all_eigenval))
    np.save(output_eigenvec,np.array(all_eigenvec))
    t2 = time()
    return t2-t1

def plots_eigenval_eigenvec(input_eigenval, input_eigenvec, output_plot_eigenval, output_plot_eigenvec):
    pt = PlotlyTransitions(is_msm=False,path_eigenvalues=input_eigenval,path_eigenvectors=input_eigenvec)
    # eigenvectors
    pt.plot_eigenvectors_flat()
    pt.save_to(output_plot_eigenvec,height=1200)
    # eigenvalues
    pt.plot_eigenvalues()
    pt.save_to(output_plot_eigenval)

In [12]:
pt_identifier = "water_water"
grid_identifier = "bigger_ideal"

In [13]:
my_shape, my_time = run_base_sqra(pt_identifier, grid_identifier, output_path=f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_pure_rate_matrix.npz")

In [14]:
print(my_shape, my_time)

(50400, 50400) 0.9223246574401855


In [15]:
dec_time = run_get_eigenval_eigenvec(f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_pure_rate_matrix.npz", 
                          f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_pure_eigenval.npy", f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_pure_eigenvec.npy")

In [16]:
print(dec_time)

114.57811141014099


In [17]:
LOWER_LIMS = [0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5]
fig = make_subplots(rows=len(LOWER_LIMS), cols=1, subplot_titles=[f"Lower_lim = {lower}" for lower in LOWER_LIMS])
for i, lower_lim in enumerate(LOWER_LIMS):
    run_cut_merge(grid_identifier, pt_identifier, f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_pure_rate_matrix.npz", lower_lim, None,
                  f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_rate_matrix_lower_{i}.npz",
                 f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_index_list_lower_{i}.npy")
    run_get_eigenval_eigenvec(f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_rate_matrix_lower_{i}.npz", 
                              f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_eigenval_lower_{i}.npy", 
                              f"{PATH_OUTPUT_AUTOSAVE}{pt_identifier}-{grid_identifier}_eigenvec_lower_{i}.npy")
    pt = PlotlyTransitions(is_msm=False,path_eigenvalues=input_eigenval,path_eigenvectors=input_eigenvec)
    pt.fig = fig
    pt.plot_eigenvectors_flat()
fig.show()

MemoryError: Unable to allocate 18.4 GiB for an array with shape (50400, 49019) and data type float64